In [6]:
import pandas as pd
import numpy as np
import sys

import warnings
import math

df=pd.read_csv('../data/data.csv.gz', compression='gzip', sep=',')
#df['timestamp'] = pd.to_datetime(df['time_stamp'], format="%Y-%m-%d %H:%M:%S").dt.tz_convert('Europe/Stockholm')
pd.to_datetime(df['time_stamp'], format="%Y-%m-%d %H:%M:%S").dt.tz_localize('CET')

0        2019-01-28 08:00:00+01:00
1        2019-01-28 08:01:00+01:00
2        2019-01-28 08:02:00+01:00
3        2019-01-28 08:03:00+01:00
4        2019-01-28 08:04:00+01:00
5        2019-01-28 08:05:00+01:00
6        2019-01-28 08:06:00+01:00
7        2019-01-28 08:07:00+01:00
8        2019-01-28 08:08:00+01:00
9        2019-01-28 08:09:00+01:00
10       2019-01-28 08:10:00+01:00
11       2019-01-28 08:11:00+01:00
12       2019-01-28 08:12:00+01:00
13       2019-01-28 08:13:00+01:00
14       2019-01-28 08:14:00+01:00
15       2019-01-28 08:15:00+01:00
16       2019-01-28 08:16:00+01:00
17       2019-01-28 08:17:00+01:00
18       2019-01-28 08:18:00+01:00
19       2019-01-28 08:19:00+01:00
20       2019-01-28 08:20:00+01:00
21       2019-01-28 08:21:00+01:00
22       2019-01-28 08:22:00+01:00
23       2019-01-28 08:23:00+01:00
24       2019-01-28 08:24:00+01:00
25       2019-01-28 08:25:00+01:00
26       2019-01-28 08:26:00+01:00
27       2019-01-28 08:27:00+01:00
28       2019-01-28 

In [ ]:
stock_delta = df['stock_id'] - df['stock_id'].shift()
time_delta = (df['timestamp'] - df['timestamp'].shift()).fillna(10000).dt.total_seconds()

In [145]:
stock_split_index=df.index[stock_delta!=0].tolist()
stock_split_index.append(len(df))

In [146]:
day_split_index=df.index[abs(time_delta)>36000].tolist()
day_split_index.insert(0, 0)
day_split_index.append(len(df))

In [147]:
stock_id_list = df['stock_id'].unique().tolist()
i = 0
for stock_index in stock_id_list:
    
    print("index: {}, id:{}".format(i, stock_index))
    i+=1

index: 0, id:101
index: 1, id:1027
index: 2, id:120
index: 3, id:139301
index: 4, id:15285
index: 5, id:160271
index: 6, id:161
index: 7, id:18634
index: 8, id:281
index: 9, id:283
index: 10, id:285
index: 11, id:300
index: 12, id:323
index: 13, id:340
index: 14, id:3524
index: 15, id:361
index: 16, id:366
index: 17, id:3966
index: 18, id:401
index: 19, id:402
index: 20, id:45
index: 21, id:46
index: 22, id:47
index: 23, id:4928
index: 24, id:5095
index: 25, id:81
index: 26, id:812
index: 27, id:819
index: 28, id:992
index: 29, id:999


In [148]:
current_stock_index = 0
time_series_list = []
time_series_all_list = []
for i in range(len(day_split_index) - 1):
    start_index = day_split_index[i]
    end_index = day_split_index[i + 1]
    day_time_series = df.iloc[start_index:end_index]
    time_series_list.append(day_time_series)
    if end_index == stock_split_index[current_stock_index + 1]:
        print("finished stock: {}, got {} time series".format(current_stock_index, len(time_series_list)))
        time_series_all_list.append(time_series_list)
        time_series_list = []
        current_stock_index += 1

finished stock: 0, got 60 time series
finished stock: 1, got 60 time series
finished stock: 2, got 60 time series
finished stock: 3, got 60 time series
finished stock: 4, got 60 time series
finished stock: 5, got 60 time series
finished stock: 6, got 60 time series
finished stock: 7, got 60 time series
finished stock: 8, got 60 time series
finished stock: 9, got 60 time series
finished stock: 10, got 60 time series
finished stock: 11, got 60 time series
finished stock: 12, got 60 time series
finished stock: 13, got 60 time series
finished stock: 14, got 60 time series
finished stock: 15, got 60 time series
finished stock: 16, got 60 time series
finished stock: 17, got 60 time series
finished stock: 18, got 60 time series
finished stock: 19, got 60 time series
finished stock: 20, got 60 time series
finished stock: 21, got 60 time series
finished stock: 22, got 60 time series
finished stock: 23, got 60 time series
finished stock: 24, got 60 time series
finished stock: 25, got 60 time ser

In [149]:
for stock_id in range(len(time_series_all_list)):
    print("handling stock_id: {}".format(stock_id))
    for day_id in range(len(time_series_all_list[stock_id])):
        # :-1 is that we don't like the last record at 17:30 which is a aggregated number.
        df = time_series_all_list[stock_id][day_id]
        # some data might miss, we must make a right join with full time series
        # and do fillna.
        df2 = df.set_index('timestamp')
        ts = df2.index.min()
        start_time_str = "{}-{:02d}-{:02d} 8:54:00".format(ts.year, ts.month, ts.day)
        start_ts = pd.Timestamp(start_time_str, tz=ts.tz)
        # periods=510 means from 9 to 17.29
        dti = pd.date_range(start_ts , periods=516, freq='min').to_series(keep_tz=True).rename('time')
        # remove from 17.25 - 17.28
        #dti.drop(dti.tail(5).head(4).index, inplace=True)
        df3 = df2.join(dti, how='right')
        if day_id == 0: # the first day, we must set the value from 8.55-8.59 as same as 9.00
            df3['last'].iloc[0] = df3['last'].iloc[5]
        else:
            df3['last'].iloc[0] = time_series_all_list[stock_id][day_id-1]['last'].iloc[-1]
        
        
        df3['last'].interpolate(method='linear', inplace=True)
        df3['volume'].iloc[:6] = df3['volume'].iloc[6] / 6
        df3['volume'].iloc[6] = df3['volume'].iloc[6] / 6
        df3['volume'].iloc[-5:] = df3['volume'].iloc[-1]/5
        
        
        df = df3.reset_index().rename({'index':'timestamp'}, axis=1)
        #df['timestamp'] = pd.to_datetime(df['time_stamp'], format="%Y-%m-%d %H:%M:%S").dt.tz_convert('Europe/Stockholm')
        df['ema_10'] = df['last'].ewm(span=10, adjust=False).mean()
        df['ema_20'] = df['last'].ewm(span=20, adjust=False).mean()
        df['diff_ema_10']=(df['ema_10'].diff()[1:]/df['ema_10']).fillna(0)
        # the first diff at 9:00 is the difference between today's open and yesterday's last.

        df['diff_ema_20']=(df['ema_20'].diff()[1:]/df['ema_20']).fillna(0)
        df['value_ema_10_beta_99'] = 0
        df['value_ema_20_beta_99'] = 0
        df['value_ema_10_beta_98'] = 0
        df['value_ema_20_beta_98'] = 0
        for iter_id in range(10):
            df['value_ema_10_beta_99'] = df['diff_ema_10'].shift(-1).fillna(0) + \
                0.99 * df['value_ema_10_beta_99'].shift(-1).fillna(0)
            df['value_ema_10_beta_98'] = df['diff_ema_10'].shift(-1).fillna(0) + \
                0.98 * df['value_ema_10_beta_98'].shift(-1).fillna(0)
            df['value_ema_20_beta_99'] = df['diff_ema_20'].shift(-1).fillna(0) + \
                0.99 * df['value_ema_20_beta_99'].shift(-1).fillna(0)
            df['value_ema_20_beta_98'] = df['diff_ema_20'].shift(-1).fillna(0) + \
                0.98 * df['value_ema_20_beta_98'].shift(-1).fillna(0)
        # drop the first row because diff is nan    
        #df.drop(0, inplace=True)
        time_series_all_list[stock_id][day_id] = df

handling stock_id: 0
handling stock_id: 1
handling stock_id: 2
handling stock_id: 3
handling stock_id: 4
handling stock_id: 5
handling stock_id: 6
handling stock_id: 7
handling stock_id: 8
handling stock_id: 9
handling stock_id: 10
handling stock_id: 11
handling stock_id: 12
handling stock_id: 13
handling stock_id: 14
handling stock_id: 15
handling stock_id: 16
handling stock_id: 17
handling stock_id: 18
handling stock_id: 19
handling stock_id: 20
handling stock_id: 21
handling stock_id: 22
handling stock_id: 23
handling stock_id: 24
handling stock_id: 25
handling stock_id: 26
handling stock_id: 27
handling stock_id: 28
handling stock_id: 29


In [156]:
time_series_all_list[0][2]

,timestamp,stock_id,time_stamp,open,high,low,last,volume,time,ema_10,ema_20,diff_ema_10,diff_ema_20,value_ema_10_beta_99,value_ema_20_beta_99,value_ema_10_beta_98,value_ema_20_beta_98
0,2019-01-30 08:55:00+01:00,NaN,NaN,NaN,NaN,NaN,80.220,25135.2,2019-01-30 08:55:00+01:00,80.220000,80.220000,0.000000,0.000000e+00,0.004704,0.003252,0.004509,0.003102
1,2019-01-30 08:56:00+01:00,NaN,NaN,NaN,NaN,NaN,80.328,25135.2,2019-01-30 08:56:00+01:00,80.239636,80.230286,0.000245,1.282024e-04,0.004803,0.003443,0.004624,0.003296
2,2019-01-30 08:57:00+01:00,NaN,NaN,NaN,NaN,NaN,80.436,25135.2,2019-01-30 08:57:00+01:00,80.275339,80.249878,0.000445,2.441354e-04,0.004441,0.003383,0.004300,0.003253
3,2019-01-30 08:58:00+01:00,NaN,NaN,NaN,NaN,NaN,80.544,25135.2,2019-01-30 08:58:00+01:00,80.324186,80.277889,0.000608,3.489337e-04,0.003944,0.003223,0.003833,0.003109
4,2019-01-30 08:59:00+01:00,NaN,NaN,NaN,NaN,NaN,80.652,25135.2,2019-01-30 08:59:00+01:00,80.383789,80.313519,0.000741,4.436314e-04,0.003213,0.002908,0.003134,0.002811
5,2019-01-30 09:00:00+01:00,101.0,2019-01-30 08:00:00+00,80.50,80.76,80.30,80.760,25135.2,2019-01-30 09:00:00+01:00,80.452191,80.356041,0.000850,5.291701e-04,0.002162,0.002386,0.002126,0.002313
6,2019-01-30 09:01:00+01:00,101.0,2019-01-30 08:01:00+00,80.64,80.64,80.60,80.600,7511.0,2019-01-30 09:01:00+01:00,80.479065,80.379275,0.000334,2.890572e-04,0.001580,0.002060,0.001585,0.002012
7,2019-01-30 09:02:00+01:00,101.0,2019-01-30 08:02:00+00,80.70,80.82,80.70,80.800,9069.0,2019-01-30 09:02:00+01:00,80.537417,80.419344,0.000725,4.982513e-04,0.000687,0.001546,0.000717,0.001516
8,2019-01-30 09:03:00+01:00,101.0,2019-01-30 08:03:00+00,80.78,80.78,80.70,80.740,24929.0,2019-01-30 09:03:00+01:00,80.574250,80.449883,0.000457,3.795986e-04,0.000129,0.001172,0.000171,0.001153
9,2019-01-30 09:04:00+01:00,101.0,2019-01-30 08:04:00+00,80.70,80.74,80.62,80.620,34816.0,2019-01-30 09:04:00+01:00,80.582568,80.466084,0.000103,2.013475e-04,0.000024,0.001017,0.000067,0.001005


In [151]:
columns = ['timestamp','last', 'diff_ema_20', 'value_ema_20_beta_99', 'value_ema_20_beta_98', 
           'diff_ema_10', 'value_ema_10_beta_99', 'value_ema_10_beta_98', 'volume']

print(time_series_all_list[0][1][columns])

                    timestamp    last   diff_ema_20  value_ema_20_beta_99  \
0   2019-01-29 08:55:00+01:00  79.020  0.000000e+00              0.000937   
1   2019-01-29 08:56:00+01:00  79.036  1.928347e-05              0.001016   
2   2019-01-29 08:57:00+01:00  79.052  3.672908e-05              0.001202   
3   2019-01-29 08:58:00+01:00  79.068  5.251108e-05              0.001442   
4   2019-01-29 08:59:00+01:00  79.084  6.678732e-05              0.001467   
5   2019-01-29 09:00:00+01:00  79.100  7.970073e-05              0.001362   
6   2019-01-29 09:01:00+01:00  79.040 -1.857971e-07              0.001274   
7   2019-01-29 09:02:00+01:00  79.260  2.648471e-04              0.000927   
8   2019-01-29 09:03:00+01:00  79.200  1.673187e-04              0.000794   
9   2019-01-29 09:04:00+01:00  79.240  1.995203e-04              0.000735   
10  2019-01-29 09:05:00+01:00  79.180  1.082563e-04              0.000710   
11  2019-01-29 09:06:00+01:00  79.180  9.793655e-05              0.000710   

In [152]:
%matplotlib notebook
from ipywidgets import *

widgets.IntSlider(min=0,max=len(time_series_all_list[0]) - 1,step=1,value=10)

@interact(stock_id=widgets.IntSlider(min=0,max=len(time_series_all_list) - 1,step=1,value=10), day_id = widgets.IntSlider(min=0,max=len(time_series_all_list[0]) - 1,step=1,value=10))
def plot(stock_id, day_id ):
    df = time_series_all_list[stock_id][day_id]
    df.plot(x='timestamp', y=['last','ema_20'])
    df.plot(x='timestamp', y=['value_ema_20_beta_98'])

interactive(children=(IntSlider(value=10, description='stock_id', max=29), IntSlider(value=10, description='da…

In [157]:

from functools import reduce

# to order the columns in the order: last1,last2...last30, diff1, diff2,...diff30, value1, value2,...value30
def func(x):
    if 'timestamp' in x:
        return 0
    num = int(x.split('_')[-1])

    if 'last' in x:
        return 100 + num
    elif 'volume' in x:
        return 150 + num
    elif 'diff_ema_10' in x:
        return 200 + num
    elif 'diff_ema_20' in x:
        return 300 + num
    elif 'value_ema_10_beta_98' in x:
        return 400 + num
    elif 'value_ema_10_beta_99' in x:
        return 500 + num
    elif 'value_ema_20_beta_98' in x:
        return 600 + num
    elif 'value_ema_20_beta_99' in x:
        return 700 + num


def merge(df1, df2):
    global index
    merged = df1.set_index('timestamp').join(df2.set_index('timestamp'), how='outer')
    index += 1

    return merged.reset_index().rename({'index':'timestamp'}, axis=1)


columns_wanted = ['timestamp','last', 'diff_ema_20', 'value_ema_20_beta_99', 'value_ema_20_beta_98', 
           'diff_ema_10', 'value_ema_10_beta_99', 'value_ema_10_beta_98', 'volume']
df_day_list = []

for day_id in range(len(time_series_all_list[0])):
    print("processing day: {}".format(day_id))
    df_list = []
    for i in range(len(stock_id_list)):
        rename_map = {'last': 'last_' + str(i),
                      'diff_ema_20':'diff_ema_20_' + str(i),
                      'value_ema_20_beta_99':'value_ema_20_beta_99_' + str(i),
                      'value_ema_20_beta_98':'value_ema_20_beta_98_' + str(i),
                      'diff_ema_10':'diff_ema_10_' + str(i),
                      'value_ema_10_beta_99':'value_ema_10_beta_99_' + str(i),
                      'value_ema_10_beta_98':'value_ema_10_beta_98_'+ str(i),
                      'volume': 'volume_' + str(i)
                     }
        df = time_series_all_list[i][day_id][columns_wanted].rename(rename_map, axis=1)
        df_list.append(df)

    index = 0
    df_merged_daily = reduce(merge, df_list)
    columns = df_merged_daily.columns.tolist()
    columns.sort(key=func)
    df_merged_sorted = df_merged_daily[columns]
    
    df_day_list.append(df_merged_sorted)
    if day_id == 0:
        df_merged = df_merged_sorted
    else:
        df_merged = df_merged.append(df_merged_sorted)

processing day: 0
processing day: 1
processing day: 2
processing day: 3
processing day: 4
processing day: 5
processing day: 6
processing day: 7
processing day: 8
processing day: 9
processing day: 10
processing day: 11
processing day: 12
processing day: 13
processing day: 14
processing day: 15
processing day: 16
processing day: 17
processing day: 18
processing day: 19
processing day: 20
processing day: 21
processing day: 22
processing day: 23
processing day: 24
processing day: 25
processing day: 26
processing day: 27
processing day: 28
processing day: 29
processing day: 30
processing day: 31
processing day: 32
processing day: 33
processing day: 34
processing day: 35
processing day: 36
processing day: 37
processing day: 38
processing day: 39
processing day: 40
processing day: 41
processing day: 42
processing day: 43
processing day: 44
processing day: 45
processing day: 46
processing day: 47
processing day: 48
processing day: 49
processing day: 50
processing day: 51
processing day: 52
pro

In [158]:

df_merged

,timestamp,last_0,last_1,last_2,last_3,last_4,last_5,last_6,last_7,last_8,...,value_ema_20_beta_99_20,value_ema_20_beta_99_21,value_ema_20_beta_99_22,value_ema_20_beta_99_23,value_ema_20_beta_99_24,value_ema_20_beta_99_25,value_ema_20_beta_99_26,value_ema_20_beta_99_27,value_ema_20_beta_99_28,value_ema_20_beta_99_29
0,2019-01-28 08:55:00+01:00,80.78,115.200,210.80,223.70,219.55,83.00,397.10,200.300,92.500,...,-0.000877,-0.000609,0.000539,-0.000770,4.730254e-04,1.477970e-04,-0.000229,-0.001397,0.000460,0.000335
1,2019-01-28 08:56:00+01:00,80.78,115.200,210.80,223.70,219.55,83.00,397.10,200.300,92.500,...,-0.000671,-0.000351,0.000924,-0.000828,5.427926e-04,2.186621e-04,-0.000148,-0.001457,0.000616,0.000499
2,2019-01-28 08:57:00+01:00,80.78,115.200,210.80,223.70,219.55,83.00,397.10,200.300,92.500,...,-0.000541,-0.000075,0.001252,-0.000881,5.851385e-04,3.668483e-04,-0.000034,-0.001493,0.000785,0.000650
3,2019-01-28 08:58:00+01:00,80.78,115.200,210.80,223.70,219.55,83.00,397.10,200.300,92.500,...,-0.000556,-0.000133,0.001529,-0.000962,5.366612e-04,4.887431e-04,0.000092,-0.001606,0.000849,0.000904
4,2019-01-28 08:59:00+01:00,80.78,115.200,210.80,223.70,219.55,83.00,397.10,200.300,92.500,...,-0.000570,-0.000104,0.001784,-0.000822,6.244797e-04,5.867350e-04,0.000227,-0.001788,0.001038,0.001175
5,2019-01-28 09:00:00+01:00,80.78,115.200,210.80,223.70,219.55,83.00,397.10,200.300,92.500,...,-0.000584,-0.000057,0.002018,-0.000726,7.272613e-04,6.629817e-04,0.000372,-0.002054,0.001238,0.001309
6,2019-01-28 09:01:00+01:00,80.64,115.600,210.80,223.70,219.10,83.02,395.70,199.600,92.540,...,-0.000268,0.000166,0.002216,-0.000640,8.993854e-04,7.332970e-04,0.000812,-0.002084,0.001390,0.001595
7,2019-01-28 09:02:00+01:00,80.62,115.800,211.10,223.00,217.80,83.11,395.20,199.600,92.500,...,0.000203,0.000680,0.002317,-0.000197,8.414606e-04,6.557668e-04,0.001135,-0.001630,0.001780,0.001772
8,2019-01-28 09:03:00+01:00,80.58,115.950,210.70,223.00,218.40,83.07,395.40,199.450,92.340,...,0.000419,0.000959,0.002305,0.000110,6.195070e-04,9.329453e-04,0.001323,-0.001323,0.002022,0.001808
9,2019-01-28 09:04:00+01:00,80.50,115.850,210.80,223.00,218.75,83.21,396.10,199.475,92.420,...,0.000357,0.000884,0.002270,0.000266,3.584804e-04,8.901921e-04,0.001388,-0.001232,0.002068,0.001718


In [159]:
columns = df_merged_sorted.columns
last_columns = []
volume_columns = []
diff_ema_10_columns = []
diff_ema_20_columns = []
value_ema_10_beta_98_columns = []
value_ema_10_beta_99_columns = []
value_ema_20_beta_98_columns = []
value_ema_20_beta_99_columns = []
for item in columns:
    if 'last' in item:
        last_columns.append(item)
    elif 'volume' in item:
        volume_columns.append(item)
    elif 'diff_ema_10' in item:
        diff_ema_10_columns.append(item)
    elif 'diff_ema_20' in item:
        diff_ema_20_columns.append(item)
    elif 'value_ema_10_beta_98' in item:
        value_ema_10_beta_98_columns.append(item)
    elif 'value_ema_10_beta_99' in item:
        value_ema_10_beta_99_columns.append(item)
    elif 'value_ema_20_beta_98' in item:
        value_ema_20_beta_98_columns.append(item)
    elif 'value_ema_20_beta_99' in item:
        value_ema_20_beta_99_columns.append(item)



In [160]:
df_merged[['timestamp'] + volume_columns + last_columns + diff_ema_10_columns + value_ema_10_beta_98_columns].to_csv('data-prep-ema10-beta98.csv')
df_merged[['timestamp'] + volume_columns + last_columns + diff_ema_10_columns + value_ema_10_beta_99_columns].to_csv('data-prep-ema10-beta99.csv')
df_merged[['timestamp'] + volume_columns + last_columns + diff_ema_20_columns + value_ema_20_beta_98_columns].to_csv('data-prep-ema20-beta98.csv')
df_merged[['timestamp'] + volume_columns + last_columns + diff_ema_20_columns + value_ema_20_beta_99_columns].to_csv('data-prep-ema20-beta99.csv')